In [1]:

!pip install opencv-python-headless scipy

import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R
from scipy import optimize

# ----------------------- Utilities -----------------------

def project_points(points_3d, K, rvec, tvec):
    rot_mat, _ = cv2.Rodrigues(rvec)
    pts_cam = (rot_mat @ points_3d.T).T + tvec
    pts_proj = (K @ pts_cam.T).T
    u = pts_proj[:, 0] / pts_proj[:, 2]
    v = pts_proj[:, 1] / pts_proj[:, 2]
    return np.vstack([u, v]).T, pts_cam[:, 2]


def add_noise(x, sigma):
    return x + np.random.normal(scale=sigma, size=x.shape)


# ----------------------- Synthetic Checkerboard -----------------------

def create_checkerboard_points(nx=8, ny=6, square=0.01):
    xs = np.arange(nx) * square
    ys = np.arange(ny) * square
    pts = np.array([[x, y, 0.0] for y in ys for x in xs], dtype=np.float32)
    return pts


def simulate_frame(points_3d, K, rvec, tvec, image_size=(480,640), noise_pix=0.5):
    pts2d, depth = project_points(points_3d, K, rvec, tvec)
    h, w = image_size
    img = np.zeros((h, w), dtype=np.uint8)
    valid = (pts2d[:,0]>=0) & (pts2d[:,0]<w) & (pts2d[:,1]>=0) & (pts2d[:,1]<h) & (depth>0)
    for p in pts2d[valid]:
        x = int(round(p[0] + np.random.normal(scale=noise_pix)))
        y = int(round(p[1] + np.random.normal(scale=noise_pix)))
        if 0 <= x < w and 0 <= y < h:
            img[y, x] = 255
    depth_map = np.zeros(image_size, dtype=np.float32)
    for (pt, d, v) in zip(pts2d, depth, valid):
        if not v: continue
        x = int(round(pt[0])); y = int(round(pt[1]))
        depth_map[y, x] = d + np.random.normal(scale=0.001)
    return img, depth_map


# ----------------------- Feature Matching (ORB placeholder) -----------------------

def extract_and_match_features(img1, img2, nfeatures=500):
    orb = cv2.ORB_create(nfeatures=nfeatures)
    k1, d1 = orb.detectAndCompute(img1, None)
    k2, d2 = orb.detectAndCompute(img2, None)
    if d1 is None or d2 is None:
        return np.empty((0,2)), np.empty((0,2))
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(d1, d2)
    matches = sorted(matches, key=lambda x: x.distance)[:200]
    pts1 = np.array([k1[m.queryIdx].pt for m in matches], dtype=np.float32)
    pts2 = np.array([k2[m.trainIdx].pt for m in matches], dtype=np.float32)
    return pts1, pts2


# ----------------------- PnP Pose from depth -----------------------

def pnp_from_depth_matches(pts1, pts2, depth1, K):
    obj_pts = []
    img_pts = []
    for p1, p2 in zip(pts1, pts2):
        x, y = int(round(p1[0])), int(round(p1[1]))
        if not(0 <= x < depth1.shape[1] and 0 <= y < depth1.shape[0]):
            continue
        z = depth1[y,x]
        if z <= 0: continue
        X = (p1[0]-K[0,2]) * z / K[0,0]
        Y = (p1[1]-K[1,2]) * z / K[1,1]
        obj_pts.append([X, Y, z])
        img_pts.append(p2)
    if len(obj_pts) < 6:
        return None, None
    obj_pts = np.array(obj_pts, dtype=np.float32)
    img_pts = np.array(img_pts, dtype=np.float32)
    success, rvec, tvec = cv2.solvePnP(obj_pts, img_pts, K, None, flags=cv2.SOLVEPNP_EPNP)
    if not success:
        return None, None
    return rvec.reshape(3,1), tvec.reshape(3,1)


# ----------------------- ICP Registration -----------------------

def icp(A, B, max_iterations=20, tolerance=1e-6):
    assert A.shape == B.shape
    src, dst = A.copy(), B.copy()
    prev_error = 0
    for i in range(max_iterations):
        dists = np.linalg.norm(src[:,None,:] - dst[None,:,:], axis=2)
        idx = np.argmin(dists, axis=1)
        nearest = dst[idx]
        centroid_A = np.mean(src, axis=0)
        centroid_B = np.mean(nearest, axis=0)
        AA = src - centroid_A
        BB = nearest - centroid_B
        H = AA.T @ BB
        U, S, Vt = np.linalg.svd(H)
        R_mat = Vt.T @ U.T
        if np.linalg.det(R_mat) < 0:
            Vt[-1,:] *= -1
            R_mat = Vt.T @ U.T
        t = centroid_B - R_mat @ centroid_A
        src = (R_mat @ src.T).T + t
        mean_error = np.mean(np.linalg.norm(src - nearest, axis=1))
        if abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error
    return R_mat, t


# ----------------------- Simple EKF Fusion -----------------------

class SimplePoseEKF:
    def __init__(self):
        self.pos = np.zeros(3)
        self.orient = R.from_quat([0,0,0,1])
        self.P = np.eye(6) * 1e-3

    def predict_imu(self, delta_pos, delta_rotvec, Q=np.eye(6)*1e-5):
        self.pos = self.pos + delta_pos
        self.orient = R.from_rotvec(delta_rotvec) * self.orient
        self.P = self.P + Q

    def update_pnp(self, rvec, tvec, R_meas_scale=1e-2):
        rot_meas, _ = cv2.Rodrigues(rvec)
        r = R.from_matrix(rot_meas)
        pos_meas = tvec.flatten()

        y_pos = pos_meas - self.pos
        dq = r * self.orient.inv()
        y_orient = dq.as_rotvec()
        y = np.hstack([y_pos, y_orient])

        R_cov = np.eye(6) * R_meas_scale
        S = self.P + R_cov
        K = self.P @ np.linalg.inv(S)
        dx = K @ y

        self.pos += dx[:3]
        self.orient = R.from_rotvec(dx[3:]) * self.orient
        self.P = (np.eye(6) - K) @ self.P

    def get_pose(self):
        return self.pos.copy(), self.orient.as_quat()


# ----------------------- DEMO PIPELINE -----------------------

def depth_to_points(depth_map, K):
    ys, xs = np.nonzero(depth_map>0)
    zs = depth_map[ys, xs]
    xs_f = (xs - K[0,2]) * zs / K[0,0]
    ys_f = (ys - K[1,2]) * zs / K[1,1]
    pts = np.vstack([xs_f, ys_f, zs]).T
    return pts


def demo_pipeline():
    print("Running IASO demo…")

    fx = fy = 800.0
    K = np.array([[fx,0,320],[0,fy,240],[0,0,1]], dtype=np.float32)

    board = create_checkerboard_points()

    rvec1 = np.zeros(3, dtype=np.float32)
    tvec1 = np.array([0,0,0.5], dtype=np.float32)

    rvec2 = np.array([0, 0.03, 0])
    tvec2 = np.array([0.005, -0.001, 0.495])

    img1, depth1 = simulate_frame(board, K, rvec1, tvec1)
    img2, depth2 = simulate_frame(board, K, rvec2, tvec2)

    pts1, pts2 = extract_and_match_features(img1, img2)
    print("Matches:", len(pts1))

    rvec_est, tvec_est = pnp_from_depth_matches(pts1, pts2, depth1, K)
    print("PnP rvec:", rvec_est.flatten(), "tvec:", tvec_est.flatten())

    ekf = SimplePoseEKF()
    ekf.pos = tvec1.copy()
    ekf.orient = R.from_rotvec(rvec1)

    imu_pos = add_noise(tvec2 - tvec1, 0.001)
    imu_rot = add_noise(rvec2 - rvec1, 0.001)

    ekf.predict_imu(imu_pos, imu_rot)
    ekf.update_pnp(rvec_est, tvec_est)

    fused_pos, fused_quat = ekf.get_pose()
    print("Fused pose:", fused_pos, fused_quat)

    pc1 = depth_to_points(depth1, K)[:200]
    pc2 = depth_to_points(depth2, K)[:200]

    R_icp, t_icp = icp(pc1, pc2)
    print("ICP done. Translation:", t_icp)
    print("Finished.")


demo_pipeline()


Running IASO demo…
Matches: 89
PnP rvec: [-2.12075341  0.15861361  1.81027601] tvec: [ 0.60096773 -0.07362728  1.5731898 ]
Fused pose: [ 0.05910551 -0.00769364  0.59280287] [-0.09690752  0.02212088  0.08219172  0.99164718]
ICP done. Translation: [-4.16333634e-17 -1.59594560e-16  0.00000000e+00]
Finished.


In [2]:
!pip install open3d


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [1]:
!pip install plotly


In [2]:
import plotly.graph_objects as go
import numpy as np

def show_point_cloud(points, camera_pose=None):
    x, y, z = points[:,0], points[:,1], points[:,2]

    fig = go.Figure(data=[
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name='Point Cloud'
        )
    ])

    # Optional: draw camera pose
    if camera_pose is not None:
        cam_origin = camera_pose[:3,3]
        fig.add_trace(go.Scatter3d(
            x=[cam_origin[0]],
            y=[cam_origin[1]],
            z=[cam_origin[2]],
            mode='markers',
            marker=dict(size=8, symbol='x'),
            name='Camera'
        ))

    fig.update_layout(
        scene=dict(aspectmode='data'),
        height=700
    )

    fig.show()

# Example
pts = np.random.rand(2000, 3)
show_point_cloud(pts)

In [4]:
import plotly.graph_objects as go
import numpy as np

def plot_icp_viewer(source, target, aligned=None, camera_pose=None):
    fig = go.Figure()

    # Source cloud (before ICP) — red
    fig.add_trace(go.Scatter3d(
        x=source[:,0], y=source[:,1], z=source[:,2],
        mode='markers',
        marker=dict(size=2, color='red'),
        name='Source (Before ICP)'
    ))

    # Target cloud — blue
    fig.add_trace(go.Scatter3d(
        x=target[:,0], y=target[:,1], z=target[:,2],
        mode='markers',
        marker=dict(size=2, color='blue'),
        name='Target'
    ))

    # Aligned cloud — green
    if aligned is not None:
        fig.add_trace(go.Scatter3d(
            x=aligned[:,0], y=aligned[:,1], z=aligned[:,2],
            mode='markers',
            marker=dict(size=2, color='green'),
            name='Aligned (After ICP)'
        ))

    # Optional camera pose
    if camera_pose is not None:
        cam_origin = camera_pose[:3, 3]  # extract XYZ
        fig.add_trace(go.Scatter3d(
            x=[cam_origin[0]],
            y=[cam_origin[1]],
            z=[cam_origin[2]],
            mode='markers',
            marker=dict(size=8, symbol='x', color='black'),
            name='Camera Pose'
        ))

    fig.update_layout(
        scene=dict(aspectmode='data'),
        height=800,
        title="IASO ICP Point Cloud Viewer (Colab-Safe)"
    )

    fig.show()


# ==== TEST EXAMPLE ====
# Remove this part when using real IASO data
source = np.random.rand(1000,3)
target = np.random.rand(1000,3)
aligned = source + np.array([0.1,0.0,0.0])  # fake ICP result

plot_icp_viewer(source, target, aligned)


In [5]:
!pip install opencv-python-headless matplotlib torch torchvision
!pip install opencv-python
!pip install plotly
!pip install timm
!pip install git+https://github.com/isl-org/MiDaS.git


  Cloning https://github.com/isl-org/MiDaS.git to /tmp/pip-req-build-ag18dfpw
  Running command git clone --filter=blob:none --quiet https://github.com/isl-org/MiDaS.git /tmp/pip-req-build-ag18dfpw
  Resolved https://github.com/isl-org/MiDaS.git to commit 454597711a62eabcbf7d1e89f3fb9f569051ac9b
ERROR: git+https://github.com/isl-org/MiDaS.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [7]:
!pip install torch torchvision
!pip install git+https://github.com/isl-org/MiDaS.git


  Cloning https://github.com/isl-org/MiDaS.git to /tmp/pip-req-build-ooff324j
  Running command git clone --filter=blob:none --quiet https://github.com/isl-org/MiDaS.git /tmp/pip-req-build-ooff324j
  Resolved https://github.com/isl-org/MiDaS.git to commit 454597711a62eabcbf7d1e89f3fb9f569051ac9b
ERROR: git+https://github.com/isl-org/MiDaS.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [9]:
# خلية 1: تثبيت المتطلبات
!pip install torch torchvision
!apt-get install -y libgl1-mesa-glx  # قد يكون مفيد إذا ظهرت أخطاء عرض لاحقاً

# خلية 2: استنساخ المستودع الرسمي لـ MiDaS من Github
%cd /content
!git clone https://github.com/intel-isl/MiDaS.git

# خلية 3: تثبيت المتطلبات من داخل المجلد MiDaS
%cd /content/MiDaS
!pip install -r requirements.txt

# خلية 4: التأكد من أن python يعرف مسار MiDaS
import sys
sys.path.append('/content/MiDaS')

# ثم استيراد MiDaS
from midas.dpt_depth import DPTDepthModel
from midas.transforms import Resize, NormalizeImage, PrepareForNet

print("تم استيراد MiDaS بنجاح!")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
/content
Cloning into 'MiDaS'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 622 (delta 183), reused 136 (delta 136), pack-reused 377 (from 2)
Receiving objects: 100% (622/622), 3.44 MiB | 9.07 MiB/s, done.
Resolving deltas: 100% (246/246), done.
/content/MiDaS
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
تم استيراد MiDaS بنجاح!


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning:

Importing from timm.models.layers is deprecated, please import via timm.layers



In [13]:
import torch

model_type = "DPT_Large"  # أو "DPT_Hybrid" أو "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning:

You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour



Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt


100%|██████████| 1.28G/1.28G [00:19<00:00, 69.6MB/s]


DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
    

In [14]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [17]:
import torch
import cv2
import numpy as np
import plotly.graph_objects as go

# 1️⃣ تحميل الموديل من cache
model_type = "DPT_Large"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# 2️⃣ تحميل التحويلات المناسبة
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type in ["DPT_Large", "DPT_Hybrid"]:
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# 3️⃣ تحميل الصورة الجراحية
img_path = "/content/Screenshot 2025-11-19 195003.png"  # ضع مسار الصورة هنا
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 4️⃣ تطبيق التحويل على الصورة
input_batch = transform(img).to(device)

# 5️⃣ توقع خريطة العمق
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False
    ).squeeze().cpu().numpy()

# 6️⃣ تحويل خريطة العمق إلى سحابة نقاط
h, w = prediction.shape
xx, yy = np.meshgrid(np.arange(w), np.arange(h))
z = prediction / np.max(prediction)
x = (xx - w/2) * z / 100
y = (yy - h/2) * z / 100
points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
colors = img.reshape(-1,3)/255.0

# تقليل عدد النقاط للعرض أسرع
points = points[::50]
colors = colors[::50]

# 7️⃣ عرض 3D Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=points[:,0], y=points[:,1], z=points[:,2],
    mode='markers',
    marker=dict(size=2, color=colors),
)])
fig.update_layout(scene=dict(aspectmode='data'),
                  height=700,
                  title="Surgical Image as 3D Point Cloud")
fig.show()


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [1]:
# ===== تثبيت المتطلبات =====
# !pip install torch torchvision opencv-python plotly open3d

# ===== استيراد المكتبات =====
import torch
import cv2
import numpy as np
import plotly.graph_objects as go
import open3d as o3d

# ===== تحميل نموذج MiDaS أصغر =====
model_type = "MiDaS_small"  # أصغر، أقل استهلاك للرام
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# التحويلات المناسبة للموديل
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type in ["DPT_Large", "DPT_Hybrid"]:
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# ===== تحميل الصورة الجراحية =====
img_path = "/content/Screenshot 2025-11-19 195003.png"  # ضع مسار الصورة هنا
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ===== إعادة تحجيم الصورة لتقليل استهلاك الذاكرة =====
scale = 0.3  # يمكن تغييرها حسب حجم الصورة
h, w = img.shape[:2]
img = cv2.resize(img, (int(w*scale), int(h*scale)))

# ===== تقدير العمق =====
input_batch = transform(img).to(device)
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False
    ).squeeze().cpu().numpy()

# ===== تحويل إلى سحابة نقاط 3D =====
h, w = prediction.shape
xx, yy = np.meshgrid(np.arange(w), np.arange(h))
z = prediction / np.max(prediction)
x = (xx - w/2) * z / 50   # تعديل scale للتقليل من حجم المشهد
y = (yy - h/2) * z / 50
points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
colors = img.reshape(-1,3)/255.0

# تقليل عدد النقاط للعرض أسرع
points = points[::100]
colors = colors[::100]

# ===== عرض 3D Plotly =====
fig = go.Figure(data=[go.Scatter3d(
    x=points[:,0], y=points[:,1], z=points[:,2],
    mode='markers',
    marker=dict(size=2, color=colors),
)])
fig.update_layout(scene=dict(aspectmode='data'),
                  height=700,
                  title="Surgical Image as 3D Point Cloud")
fig.show()

# ===== خيار: تصدير السحابة إلى ملف PLY =====
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_point_cloud("surgical_scene.ply", pcd)

print("تم حفظ السحابة في 'surgical_scene.ply'")


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/rwightman/gen-efficientnet-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_lite3-b733e338.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_lite3-b733e338.pth
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v2_1/midas_v21_small_256.pt" to /root/.cache/torch/hub/checkpoints/midas_v21_small_256.pt


100%|██████████| 81.8M/81.8M [00:00<00:00, 258MB/s]
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


تم حفظ السحابة في 'surgical_scene.ply'


In [3]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# تحويل النقاط إلى Mesh باستخدام Ball Pivoting أو Poisson
pcd.estimate_normals()
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)[0]

o3d.visualization.draw_geometries([mesh])

[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [4]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(
    x=points[:,0], y=points[:,1], z=points[:,2],
    mode='markers',
    marker=dict(size=2, color=colors),
)])
fig.show()


In [5]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_point_cloud("surgical_scene.ply", pcd)


True

In [7]:
import open3d as o3d

# تحويل النقاط والألوان إلى PointCloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# تقدير normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

# إنشاء Mesh باستخدام Poisson
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)

# حفظ Mesh
o3d.io.write_triangle_mesh("surgical_scene_mesh.ply", mesh)
print("تم إنشاء Mesh وحفظه في 'surgical_scene_mesh.ply'")

تم إنشاء Mesh وحفظه في 'surgical_scene_mesh.ply'


In [9]:
# ===== تثبيت المتطلبات =====
# !pip install torch torchvision opencv-python plotly open3d

# ===== استيراد المكتبات =====
import torch
import cv2
import numpy as np
import plotly.graph_objects as go
import open3d as o3d
from google.colab import files

# ===== رفع الصورة من الجهاز =====
uploaded = files.upload()  # ستفتح نافذة لرفع الصورة
for fname in uploaded.keys():
    img_path = fname
print(f"تم رفع الصورة: {img_path}")

# ===== تحميل نموذج MiDaS دقيق =====
model_type = "DPT_Hybrid"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# التحويلات المناسبة للموديل
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type in ["DPT_Large", "DPT_Hybrid"]:
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# ===== تحميل الصورة =====
img = cv2.imread(img_path)
if img is None:
    raise ValueError("الصورة غير موجودة! تحقق من المسار والاسم.")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ===== إعادة تحجيم الصورة لتقليل استهلاك الذاكرة =====
scale = 0.5  # يمكن تغييره حسب حجم الصورة
h, w = img.shape[:2]
img = cv2.resize(img, (int(w*scale), int(h*scale)))

# ===== تقدير العمق =====
input_batch = transform(img).to(device)
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False
    ).squeeze().cpu().numpy()

# ===== تحويل إلى سحابة نقاط 3D =====
h, w = prediction.shape
xx, yy = np.meshgrid(np.arange(w), np.arange(h))
z = prediction / np.max(prediction)
x = (xx - w/2) * z / 50
y = (yy - h/2) * z / 50
points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
colors = img.reshape(-1,3)/255.0

# ===== تقليل عدد النقاط للعرض فقط =====
points_display = points[::100]
colors_display = colors[::100]

# ===== عرض 3D Plotly =====
fig = go.Figure(data=[go.Scatter3d(
    x=points_display[:,0], y=points_display[:,1], z=points_display[:,2],
    mode='markers',
    marker=dict(size=2, color=colors_display),
)])
fig.update_layout(scene=dict(aspectmode='data'),
                  height=700,
                  title="Surgical Image as 3D Point Cloud")
fig.show()

# ===== تحويل Point Cloud إلى Open3D =====
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# ===== تقدير normals قبل إنشاء Mesh =====
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))

# ===== إنشاء Mesh باستخدام Poisson =====
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)
o3d.io.write_triangle_mesh("surgical_scene_mesh.ply", mesh)

print("تم حفظ Mesh في 'surgical_scene_mesh.ply'")


Saving Screenshot 2025-11-19 195003.png to Screenshot 2025-11-19 195003 (1).png
تم رفع الصورة: Screenshot 2025-11-19 195003 (1).png


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


تم حفظ Mesh في 'surgical_scene_mesh.ply'


In [11]:
# ===== تثبيت المتطلبات =====
# !pip install torch torchvision opencv-python plotly open3d

# ===== استيراد المكتبات =====
import torch
import cv2
import numpy as np
import plotly.graph_objects as go
import open3d as o3d
from google.colab import files

# ===== رفع الصورة من الجهاز =====
uploaded = files.upload()
for fname in uploaded.keys():
    img_path = fname
print(f"تم رفع الصورة: {img_path}")

# ===== تحميل نموذج MiDaS عالي الدقة =====
model_type = "DPT_Large"  # أعلى دقة متاحة
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)

# التحويلات المناسبة للموديل
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type in ["DPT_Large", "DPT_Hybrid"]:
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# ===== تحميل الصورة =====
img = cv2.imread(img_path)
if img is None:
    raise ValueError("الصورة غير موجودة! تحقق من المسار والاسم.")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ===== إعادة تحجيم الصورة لتقليل استهلاك الذاكرة =====
scale = 0.7  # يمكن تعديلها حسب حجم الصورة والرام
h, w = img.shape[:2]
img = cv2.resize(img, (int(w*scale), int(h*scale)))

# ===== تقدير العمق =====
input_batch = transform(img).to(device)
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False
    ).squeeze().cpu().numpy()

# ===== تحسين الخريطة لتقليل التشويش =====
depth_smooth = cv2.bilateralFilter(prediction.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)

# ===== تحويل إلى سحابة نقاط 3D =====
h, w = depth_smooth.shape
xx, yy = np.meshgrid(np.arange(w), np.arange(h))
z = depth_smooth / np.max(depth_smooth)
x = (xx - w/2) * z / 50
y = (yy - h/2) * z / 50
points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
colors = img.reshape(-1,3)/255.0

# ===== تقليل عدد النقاط للعرض فقط =====
points_display = points[::100]
colors_display = colors[::100]

# ===== عرض 3D Plotly تفاعلي =====
fig = go.Figure(data=[go.Scatter3d(
    x=points_display[:,0], y=points_display[:,1], z=points_display[:,2],
    mode='markers',
    marker=dict(size=2, color=colors_display),
)])
fig.update_layout(scene=dict(aspectmode='data'),
                  height=700,
                  title="Surgical Image as 3D Point Cloud")
fig.show()

# ===== تحويل Point Cloud إلى Open3D =====
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# ===== تقدير normals محسّن قبل إنشاء Mesh =====
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=50))

# ===== إنشاء Mesh باستخدام Poisson =====
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)
o3d.io.write_triangle_mesh("surgical_scene_mesh1.ply", mesh)

print("✅ تم حفظ Mesh في 'surgical_scene_mesh.ply' مع دقة محسنة")


Saving Screenshot 2025-11-19 195003.png to Screenshot 2025-11-19 195003 (3).png
تم رفع الصورة: Screenshot 2025-11-19 195003 (3).png


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


✅ تم حفظ Mesh في 'surgical_scene_mesh.ply' مع دقة محسنة
